# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [4]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [5]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/home/lande/anaconda3/envs/llm_train/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 7/7 [00:04<00:00,  1.40it/s]
/home/lande/anaconda3/envs/llm_train/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [8]:
input_text = "解释下乾卦是什么？"

In [9]:
response, history = base_model.chat(tokenizer, query=input_text)

In [10]:
print(response)

乾卦是八卦之一，代表天、阳气、强健、刚毅等象征。乾卦是由两个阴爻夹一个阳爻构成，象征着天象的运行和阳刚之气的流行。在八宫图（后天八卦）中，乾卦位于北方，与事业、领导、刚强、自信等有关。乾卦所代表的力量积极向上，具有领导才能和决断力，能够克服困难和挑战，取得成功。在道家思想中，乾卦与“道”相联系，代表着宇宙的运行和自然法则。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [11]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，也是后天的八卦之一。它由两个阳爻夹一个阴爻构成，象征着天平之下的诉讼、争端和矛盾。在八宫图（后天八卦）中，讼卦位于西北方，与法律、诉讼、争斗、疾病等有关。讼卦所代表的力量平衡，具有公正、公平、客观的特点，能够协调双方的矛盾，达到公正的结果。在周易中，讼卦的建议是“利用动刚”，即在争端中要利用阳刚之气的优势，同时要注意保持中立的立场，以达到和解的目的。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

#### 加载 QLoRA Adapter(Epoch=30, Overfit, handmade-dataset)

In [13]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
step = 'checkpoint-30'
timestamp = "20250329_212843_" + step

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}/{step}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [14]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [15]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，由六个阳爻组成，象征着天。它代表刚健、健行、刚健不屈的意境。在这个卦中，下卦是乾卦，上卦是坤卦。下卦的六爻都是阳爻，表示刚健不屈的精神，而至上卦的六爻都是阴爻，表示柔和 flexibility。因此，整体上乾卦代表刚健不屈的品性，但也有一些 flexibility的特质。

乾卦的核心哲学是：天行健，君子自强不息。在事业上，乾卦代表刚健不屈、刚毅不屈的品性，但也需要柔顺、 flexibility、弹性，才能取得成功。在经商中，乾卦代表刚健不屈的毅力，但也需要坚定信念和灵活调整。在感情方面，乾卦代表刚健不屈的品性，但也需要柔情似水。

乾卦的卦象是由两个卦相叠而成：下卦是乾卦，上卦是坤卦。下卦的六爻都是阳爻，表示刚健不屈的精神，而至上卦的六爻都是阴爻，表示柔和灵活。因此，整体上乾卦代表刚健不屈的品性，但也有一些灵活的特质。在周易中，阳爻代表阳刚之极，阴爻代表阴柔之极。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250329_212843_checkpoint-30）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [16]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250329_212843_checkpoint-30）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是由坎卦和坤卦相叠而成，代表着诉讼和纷争。在周易中，它预示着 conflict（争斗）会带来更多的问题，但也可以通过适当的解决方案来化解。

讼卦的核心哲学是：争斗引起争斗，止争斗才能化解矛盾。因此，它强调止争斗、求和化解的重要性。周易中，止战之戈（戈）象征着 stop（停止）和止战之戈的和平 symbol，这表明在战争和争斗中，应该停止开战，寻求和平解决方案。

讼卦的运势是：上卦为止战之戈，下卦为水火相克。整体来看，这个卦提示你在生活中会面临矛盾和纷争，需要停止开战，寻求和平化解。然而，止战之戈也象征着和平 symbol，说明有和平解决的可能。

讼卦的运势解释是：止战之戈，寓意着停止争斗，寻求和平化解。综述止争斗、止战之戈以及和平解决，都在提示你寻求和平，避免战争和争斗，以解决矛盾。

讼卦的卦象是由坎卦和坤卦相叠而成，坎卦象征水，坤卦象征地。整体来看，表示 conflict（争斗）引发地面的矛盾，需要化解。因此，这个卦提示你面临争斗和矛盾，需要停止开战，寻求和平化解。

在 business（商业）方面，止战之戈暗示着应该停止战争和争斗，而寻求和平解决。这表明商业活动中可能会面临矛盾和纷争，需要停止开战，寻求和平化解。商业决策者应该停止战争，寻求和平合作，以解决商业矛盾。

在恋爱方面，止战之

In [17]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
{'name': '师卦是由坎卦（水）和坤卦（地）相叠而成。在周易中，它代表着军队的力量和军队领导者的形象。象辞中描述了大地孕育着万物，预示着大地即将 shake（ shake is a verb that means to shake or vibrate vigorously, often as a result of an explosion or impact）。', 'content': '\n师卦的核心观念是：大地孕育着万物，预示着大地即将 shake（摇动或振动），大地之摇动 shake 比喻强大力量的作用，同时也预示着大地即将恢复平静。师卦的核心观念是：大地孕育着万物，预示着大地即将 shake（摇动或振动），大地之摇动 shake 比喻强大力量的作用，同时也预示着大地即将恢复平静。\n\n师卦的吉凶观瞻在于慎用兵戈，及时止损，以免损失惨重。得利者，预言其家族繁荣，事业有成；不得利者，则警告其轻率行事，以免遭灾。然而，周易并非完全乐观，也警惕着过度自信和轻敌。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250329_212843_checkpoint-30）：
[gMASK]sop 师卦是什么？ 师卦是由坎卦（水）和坤卦（地）相叠而成。它象征着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


#### 加载 QLoRA Adapter(Epoch=20, Overfit, handmade-dataset)

In [22]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
step = 'checkpoint-20'
timestamp = "20250329_212843_" + step

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}/{step}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_20 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [23]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_20, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦中的第一卦，由六个阳爻组成，形成一个刚健、充满活力和充满生机的卦象。在周易中，乾卦象征着天，代表着宇宙的运行和自然界的法则。乾卦所代表的是刚健、充满活力和充满生机。

乾卦的意义非常丰富，它既象征着天，也象征着地面。在周易中，乾卦代表着宇宙的运行和自然界的法则，同时也象征着君子应当效仿天行的刚健和正道。在人事中，乾卦象征着君子应当坚定信念，努力奋斗，充满自信和勇气，具有权威和领导力。

乾卦的核心哲学是刚健、充满生机和充满活力，它代表着一种坚定不移、勇敢前进的精神。因此，乾卦鼓励人们坚定信念，努力奋斗，充满自信和勇气，具有权威和领导力。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250329_212843_checkpoint-20）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通，行健，正固。得此卦者，吉利。然而，也有可能会遇到挫折和困难。但总体而言，这是一个吉祥之卦。

在事业方面，乾卦提示你要有坚定的信念和决心，努力奋斗，刚健不屈，才能取得成功。对于经商，乾卦提示你要有勇气和自信，刚健不屈，预测准确，勇于决策，从而获得利的机遇。在婚姻方面，乾卦提示你要坚定信念，忠诚专一，坚持正义，才能获得幸福。

此外，乾卦也适用于其他方面，如预测、经商、经商、决策等。在不同的情境下，乾卦所代表的含义也会有所不同。


In [24]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_20, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是一个虚构的卦象，它并不在《易经》或其他正统的卦象中出现。这个卦象是由两个相反的卦象组合而成，一个是地卦，代表大地之厚实和承载，另一个是水卦，代表流动性、润泽和流动。在这个卦象中，地卦和水源相互依存，形成一种相互支持的关系。

在哲学上，地水师卦代表大地之厚实和承载，以及水流之润泽和流动。它象征大地之厚实和承载可以带来灵活性和滋润。在决策中，它预示着大地之厚实和承载将带来好运，但同时也需要警惕水流可能带来的灾害。

地水师卦并不是一个正式的卦象，因此，它并没有固定的 interpreter。然而，在某些解梦和卜卦的实践中，人们可能会用这个卦象来象征大地之厚实和承载，以及水流之润泽和流动。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250329_212843_checkpoint-20）：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由三个卦相叠组成的卦象，分别是下卦乾为天，中间卦坎为水，上卦坤为地。这个卦象预示着天地相映成趣，地地相融，具有吉祥之象。

在周易中，地水师卦解读为“主客相待，有待于也”。这句话意味着，在这个卦象中，主人（主方）与客人（客方）相互等待，都期待着行动，因此形成一种相互等待、观望的局面。

地水师卦的哲学内涵是“天地相成，主客相待”，预示着事物的发展变化需要时间，需要耐心等待。同时，也提示人们在行动之前要充分做好准备，掌握主动权，以避免不利后果。

然而，地水师卦并非完全吉祥，也预示着潜在的危险。因此，在决策时，需要根据实际情况做出判断，权衡利弊，避免冲动行事。在一般情况下，地水师卦预示着顺利，但也有潜在的危险，需要谨慎对待。


In [25]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_20, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个卦象，由上卦坎（水）和下卦震（雷）组成。坎象征云，震象征雷，云行雷动，寓意着和平与和解。然而，由于云和雷的本性不同，它们在卦象中相互制约，因此形成了一个卦象。

在周易的哲学中，每个卦象都代表着一种特定的卦辞，用来指导人们如何应对生活中的各种情况。讼卦的卦辞提醒人们，在处理纷争和诉讼时，需要诉诸于理性和公平，避免过度愤怒和暴力。

尽管讼卦预示着和平与和解，但也不能完全排除争端和矛盾的可能性。因此，在应对讼卦时，需要根据实际情况灵活应对，尽量通过和平谈判和协商来解决问题，避免过度执着于自身利益，导致矛盾升级。

总体来说，讼卦预示着如果在处理争端和诉讼时能够诉诸于理性和公平，就能得到圆满的结果。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250329_212843_checkpoint-20）：
[gMASK]sop 周易中的讼卦是什么? 在周易中，讼卦是一个充满警示的卦象。它由上卦乾和下卦坎组成，代表着天雷和行动。意味着，当君子们处理纷争和是非时，应当积极行动，以诚实和公正的态度去解决问题。然而，由于止于至善，因此君子们也应止于至善，避免止于至恶，以免招致祸端。

讼卦的卦辞中提到：“天雷行地，雷震雷鸣，君子涉水行舟，涉水行舟，依赖他人，君子敬德莫敢怀罪。”它告诉我们，处理纷争和是非应当积极行动，同时要诚实公正，依赖他人，表示信任和合作。

在事业上，讼卦表示在事业中可能会遇到一些困难和挫折，但只要坚持诚实公正的原则，积极解决纷争，依赖合理的策略和合作，就能获得美好的结果。在经商中，象征着商贾们应当遵守诚信，避免与不良商家交往，否则会招致不幸。

总体来说，讼卦提示我们，在处理纷争和冲突时，要积极面对，诚实公正，依赖他人，避免止于至恶。


#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [26]:
from peft import PeftModel, PeftConfig

step = 'checkpoint-50'
timestamp = "20250329_212843_" + step

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}/{step}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [27]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。在这个卦中，预示着positivity 

乾卦的核心哲学是：天行健，君子擬之。君子要像天行健一样，坚健不屈，运行不已，自强不息。乾卦不仅仅是一个卦象，同时也是一种卦象观，一种哲学象征。在周易中，乾卦是大吉大利的卦象，代表着吉顺亨通。然而，它也预示着困难与挫折。因此，针对不同的情况，需要采取不同的措施，既要坚持正义，也要有灵活的变通。

在事业方面，乾卦预示着事业发展顺境，但也提醒我要光明磊落，刚健不屈，不懈努力。在经商方面，乾卦提示我要坚定信念，努力奋斗，不屈不挠，追求长远，dsyz。在求名方面，乾卦提醒我要修养道德，追求卓越，光明磊落，树立良好的声誉。

在婚恋方面，乾卦提示我要坚定信念，努力奋斗，追求长远，但也提醒我要光明磊落，修养道德，维护和谐。在决策方面，乾卦提示我要修养道德，坚定信念，追求卓越，追求长远。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [28]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
抱歉，您的问题不太清晰明确，无法确定您所指的“地水师卦”是指什么。在周易中，共有64卦，每一卦都由两个卦相组合而成，形成一个完整的哲学体系。如果您能提供更多的信息，我将尽力为您解答。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦



卦象：雷山卦

卦辞：观雷山，摇摇欲坠，临川局，鸟兽惊，川山相耀，龙腾虎跃。

象辞：象辞：山中有风，谓之劳卦。劳卦劳苦，不可贞谋，观市招携， setting the market.

卦象：地山卦

卦辞：地山卦，主客皆忧，欲求其利，不可强求。

像山中有风，这是观象之辞，说明卦象的形势。

卦象：地山卦

地山卦是卦象之一，由下卦坎和上卦坤组成。坎卦象征云，坤卦象征地。云地之间，意味着云彩遮盖大地，大地又有地山之象。

地山卦的卦象象征著，劳苦且摇摇欲坠的状况。这是观摇之卦，摇摇欲坠，不可获全，观市招携， setting the market。

地山卦的卦辞是：观雷山，摇摇欲坠，临川局，鸟兽惊，川山相耀，龙腾虎跃。

地山卦的象辞是：象辞，山中有风，谓之劳卦。劳卦劳苦，不可贞谋，观市招携， setting the market。

地山卦的卦象之象是，地中有云，象征卦象摇摇欲坠的状况。


In [29]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦是一个充满警示的卦象，它由上卦乾（天）和下卦坎（水）组成。虽然卦象看似和谐，但预示着前进 with caution，必须谨慎行事。因为虽然 Conflict (争端)存在，但也有 resolution (解决)的可能。

在这个卦中，卦象是由天与水相交而成，代表着天行覆盖大地，显示了强大的力量。然而，也由于卦象中下卦为坎，代表着河川，这意味着困难、挫折和困境，需要谨慎行事，谋宜解决。

对于求签者而言，卦象提示必须谨慎行事，宜谋 solutions 寻求解决办法。即使事务进展顺利，也应谨慎行事，恐惹事端。对于求财者，虽然有机会得到财，但需要谨慎投资，必须选择正确的赚钱方法。对于求名者，建议行事光明磊落，严格依法行事，不能靠运气。对于求婚者，需要决策谨慎，不可过于草率。对于求职者，需要耐心等待机会，不能急功近利。

总体而言，天水讼卦提醒我们， Conflict 存在，但也有 resolution的可能。对于任何事态，都应采取积极、谨慎、耐心和灵活的态度。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 坎坎而上下，劳劳而动。 君子观此，忧愁而不忧，愁煎而不煎。 忧愁而不忧，愁煎而不煎，此之谓大通。 天地之间，大通之象也。 观于物者，其象如此。 观于物者，其象如此，则其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间，物极而通。 天地之间，物极而通，此其道至矣。 天地之间